# Sensor Management Example

In [2]:
import numpy as np
import random
from ordered_set import OrderedSet
from datetime import datetime, timedelta
from mpar_sim.radar import PhasedArrayRadar
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

## Simulate targets

In [12]:
from datetime import datetime
from datetime import timedelta

import numpy as np
from ordered_set import OrderedSet

from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
    ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

n_steps = 50
start_time = datetime.now()
truths = OrderedSet()  # Truths across all time
current_truths = set()  # Truths alive at current time

transition_model = CombinedLinearGaussianTransitionModel([
    ConstantVelocity(0.05),
    ConstantVelocity(0.05),
    ConstantVelocity(0.0),
])

for k in range(n_steps):
  # Death
  for truth in current_truths.copy():
    if np.random.rand() <= 0.01:  # Death probability
      current_truths.remove(truth)
  # Update truths
  for truth in current_truths:
    truth.append(GroundTruthState(
        transition_model.function(truth[-1],
                                  noise=True,
                                  time_interval=timedelta(seconds=1)), timestamp=start_time + timedelta(seconds=k)))
    truth[-1].rcs = 10
  # Birth
  for _ in range(np.random.poisson(0.8)): # Birth probability
    x, y = initial_position = np.random.rand(2) * [5000, 2000]
    y -= 500
    x_vel, y_vel = (np.random.rand(2))*2 - 1 # Range [-1, 1] for x and y velocity
    state = GroundTruthState([x, x_vel, y, y_vel, 0, 0], 
                             timestamp=start_time + timedelta(seconds=k))
    state.rcs = 10
    # Add to truth set for current and all timestamps
    truth = GroundTruthPath([state])
    current_truths.add(truth)
    truths.add(truth)

Plot ground truths using the Plotterly class

In [4]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
plotter.plot_ground_truths(truths, [0, 2])
plotter.fig

## Sensor setup

In [5]:
from mpar_sim.radar import PhasedArrayRadar
from stonesoup.types.state import StateVector

radar = PhasedArrayRadar(
  position=np.array([0, 0, 0]),
  rotation_offset=StateVector(np.array([0, 0, 0])),
  position_mapping=(0, 2, 4),
)
radar.timestamp = start_time

## Run the Simulation

In [6]:
from mpar_sim.look import RadarLook
import math

# Select action(s)
look = RadarLook(
    start_time=start_time,
    azimuth_beamwidth=10,
    elevation_beamwidth=1,
    tx_power=100,
    azimuth_steering_angle=0,
    elevation_steering_angle=0,
    bandwidth=500e6,
    pulsewidth=1e-6,
    prf=5e3,
    n_pulses=10,
)
# Schedule looks
radar.load_look(look)

all_measurements = []
for k in range(n_steps):
  measurements = set()
  timestamp = start_time + timedelta(seconds=k)
  
  # Collect all 
  current_truths = OrderedSet()
  for truth in truths:
    try:
      truth_state = truth[timestamp]
    except IndexError:
      # Truth is not alive at this time
      continue
    current_truths.add(GroundTruthPath([truth_state]))
    
  measurements |= radar.measure(current_truths, noise = True)
  all_measurements.append(measurements)


In [7]:
# Plot true detections
plotter.plot_sensors(radar, "Radar")
plotter.plot_measurements(all_measurements, [0, 2])
plotter.fig

## Create Tracker Components

Create tracker

In [8]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import ExtendedKalmanUpdater
updater = ExtendedKalmanUpdater(measurement_model=None)

from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis, Euclidean
hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=50)

from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator = GNNWith2DAssignment(hypothesiser)

Create deleter

In [9]:
from stonesoup.deleter.time import UpdateTimeStepsDeleter
deleter = UpdateTimeStepsDeleter(5)

Create an initiator

In [10]:
from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator, SimpleMeasurementInitiator
initiator = MultiMeasurementInitiator(
    prior_state=GaussianState([0, 0, 0, 0, 0, 0], np.diag([0, 0, 0, 0, 0, 0])),
    measurement_model=None,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
    min_points=3,
)

## Run the tracker

In [11]:
tracks, all_tracks = set(), set()

for n, measurements in enumerate(all_measurements):
  # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
  hypotheses = data_associator.associate(tracks,
                                         measurements,
                                         start_time + timedelta(n))
  associated_measurements = set()
  for track in tracks:
    hypothesis = hypotheses[track]
    if hypothesis.measurement:
      post = updater.update(hypothesis)
      track.append(post)
      associated_measurements.add(hypothesis.measurement)
    else:  # When data associator says no detections are good enough, we'll keep the prediction
      track.append(hypothesis.prediction)

  # Carry out deletion and initiation
  tracks -= deleter.delete_tracks(tracks)
  tracks |= initiator.initiate(measurements - associated_measurements,
                               start_time + timedelta(n))
  all_tracks |= tracks


/home/shane/.cache/pypoetry/virtualenvs/mpar-sim-H1aD6MoI-py3.10/lib/python3.10/site-packages/stonesoup/measures.py:246: RuntimeWarning:

invalid value encountered in sqrt



Plot the resulting tracks

In [11]:
plotter.plot_tracks(all_tracks, [0,2])
plotter.fig.show()